In [8]:
#from PyPDF2 import PdfReader
import PyPDF2
import pandas as pd
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [42]:
prompt = f"""
I am a data analyst who works for an electric company.  My company has tariffs setup for pricing in ```{state}``` 
Read through the text below and return a summary Structured as below if the data is found:

"<Section>
    - <Category>: 
        <Summary by category>"

Here are the categories by section and what should be summarized in each category:

Territory
    - Counties: list of counties

Cost Structure
    - Rates: a bulleted list of "<rate name>:<cost>"
        - Seasonal - a bulleted list of seasonal changes in rate
        - Time of Use - a bulleted list of Time of Use (TOU)
        - Incentives - ie renewable, saver switch, demand response programs, etc.
    - Taxes and Fees 
    - Measurement - how the data will be measured by the meter.

Electric Market
    - Tariff Type - ie transmission, generation, distribution, surcharges
    - Demand type - ie commercial, industrial or residential

Regulatory
    - Regulatory - compliance

The text is
```{text}```
"""

In [30]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [ ]:

# Define the get_completion function
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,  # This is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

# Define a function to process a chunk of pages
def process_chunk(chunk):
    chunk_text = ""
    for page in chunk:
        text = page.extractText()
        chunk_text += text
    # Call the get_completion function with the chunk text
    completion = get_completion(chunk_text)
    return completion

# Open the PDF file in read-binary mode
with open(psco_tar, 'rb') as pdf_file:
    pdf_reader = PyPDF2.PdfFileReader(pdf_file)
    total_pages = len(pdf_reader.pages)
    chunk_size = 50  # Set the number of pages per chunk
    response_string = ""

    for start_page in range(0, total_pages, chunk_size):
        end_page = min(start_page + chunk_size, total_pages)

        # Create a list to store pages in the current chunk
        chunk = []
        for page_num in range(start_page, end_page):
            page = pdf_reader.getPage(page_num)
            chunk.append(page)

        # Call the processing function for the current chunk and append the result
        completion = process_chunk(chunk)
        response_string += completion

# Close the PDF file
pdf_file.close()

# The final result is stored in the response_string variable
print(response_string)


In [43]:

# Define the get_completion function
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,  # This is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

# Define a function to process a chunk of pages
def process_chunk(chunk):
    chunk_text = ""
    for page in chunk:
        text = page.extract_text()
        chunk_text += text
    # Call the get_completion function with the chunk text
    completion = get_completion(chunk_text)
    return completion

# Open the PDF file in read-binary mode
with open(psco_tar, 'rb') as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    total_pages = 10
    chunk_size = 5  # Set the number of pages per chunk
    response_string = ""

    for start_page in range(0, total_pages, chunk_size):
        end_page = min(start_page + chunk_size, total_pages)

        # Create a list to store pages in the current chunk
        chunk = []
        for page_num in range(start_page, end_page):
            page = pdf_reader.pages[page_num]
            chunk.append(page)

        # Call the processing function for the current chunk and append the result
        completion = process_chunk(chunk)
        response_string += completion

# Close the PDF file
pdf_file.close()

# The final result is stored in the response_string variable
print(response_string)


Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)

In [39]:
print(response_string)

The document is an index of the electric tariff for Public Service Company of Colorado. It includes a table of contents, territory served, general definitions, electric sales rates, electric sales service rates, commercial and industrial service rates, area lighting rates, public street and highway lighting service rates, optional service rates, electric rate adjustments, rules and regulations, and small power production and cogeneration facility policy. The document also includes contact information for Public Service Company of Colorado.This document is an advice letter from Public Service Company of Colorado, also known as Xcel Energy, regarding electric rates and services. It includes a table of contents listing the different sections and rates available. The document also provides information on the territory served by the company, including cities, towns, and communities in Colorado. The sheet numbers indicate revisions and changes made to the document over time.
